In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import scipy.stats as st
import statsmodels.api as sm
import numpy as np

from VaR_func import *
from tests import *

In [7]:
data = pd.read_csv(".\\data\\ropa.csv", index_col='Date', header=0)
data['returns'] = data.interpolate(type='linear').pct_change()
data.index = pd.to_datetime(data.index)
r = list(data['returns'])
r = r[1:]
r_test = r[-298:]
fitted_dist = pd.read_csv('fitted_distributions_best.csv', index_col=0)

In [8]:
dist_type = "t"
filename = './data_distfit/fitted_{}.csv'.format(dist_type)
df1 = pd.read_csv(filename, index_col=0)
df1

t_min  t_max                                             params  \
-1        0    798  (6.080803251126445, 0.0015888318874874476, 0.0...   
 0        0    499  (5.360464598578685, 0.002028062353721335, 0.02...   
 1        1    500  (5.321771569906347, 0.0021115908063192882, 0.0...   
 2        2    501  (5.288460197098301, 0.002026474527667888, 0.02...   
 3        3    502  (5.241928798602011, 0.002006924283874679, 0.02...   
...     ...    ...                                                ...   
 293    293    792  (18.46934189425143, 0.00019678092206142114, 0....   
 294    294    793  (17.770366010399627, 0.00012607907540755143, 0...   
 295    295    794  (17.73884167806812, 0.0001625987601562807, 0.0...   
 296    296    795  (17.731184434662175, 0.00015986835169561135, 0...   
 297    297    796  (17.219018365036497, 0.00019264723696027725, 0...   

           SSE                                     ecdf_quantiles    VaR_95  \
-1    0.131604  [-1.30628881e-01 -1.21261119e-01 -8.23572838e-...  0.040322   
 0    0.098780  [-0.1306288805140844, -0.12126111859385258, -0...  0.044278   
 1    0.105857  [-0.1306288805140844, -0.12126111859385258, -0...  0.044344   
 2    0.101235  [-0.1306288805140844, -0.12126111859385258, -0...  0.044216   
 3    0.107570  [-0.1306288805140844, -0.12126111859385258, -0...  0.044091   
...        ...                                                ...       ...   
 293  0.141359  [-0.08235728377794271, -0.07925833692240081, -...  0.037519   
 294  0.139425  [-0.08235728377794271, -0.07925833692240081, -...  0.037363   
 295  0.147290  [-0.08235728377794271, -0.07925833692240081, -...  0.037403   
 296  0.145779  [-0.08235728377794271, -0.07925833692240081, -...  0.037399   
 297  0.141755  [-0.08235728377794271, -0.07925833692240081, -...  0.037390   

        VaR_99   EVaR_95   EVaR_99  
-1    0.064102  0.029600  0.047731  
 0    0.071661  0.032868  0.053648  
 1    0.071808  0.032957  0.053794  
 2    0.071732  0.032857  0.053730  
 3    0.071651  0.032784  0.053686  
...        ...       ...       ...  
 293  0.055073  0.026319  0.040516  
 294  0.054965  0.026200  0.040414  
 295  0.055011  0.026240  0.040457  
 296  0.055008  0.026236  0.040454  
 297  0.055049  0.026249  0.040489  

[299 rows x 9 columns]

In [9]:
def summary_backtesting(df, pr=True):
    v95 = df['VaR_95'][1:]
    v99 = df['VaR_99'][1:]
    ev95 = df['EVaR_95'][1:]
    ev99 = df['EVaR_99'][1:]
    I_95 = []
    I_99 = []
    for i in range(len(r_test)):
        I_95.append(v95[i] < r_test[i])
        I_99.append(v99[i] < r_test[i])
    t = '       '
    res = {}
    res['Ku95'] = kupiec(np.array(I_95), .95)
    res['Ku99'] = kupiec(np.array(I_99), .99)
    if sum(I_95) != 0:
        res['Ch95'] = christoffersen(np.array(I_95))
    else: res['Ch95']= None
    if sum(I_99) != 0:
        res['Ch99'] = christoffersen(np.array(I_99))
    else: res['Ch99']= None
    res['Sc95'] = np.mean(scoring_quantile(r_test, v95, .95))
    res['Sc99'] = np.mean(scoring_quantile(r_test, v99, .99))
    res['ESc95'] = np.mean(scoring_expectile(r_test, ev95, .95))
    res['ESc99'] = np.mean(scoring_expectile(r_test, ev99, .99))
    
    if pr:
        print('VaR_95:')
        print(t,' Kupiec:', res['Ku95'])
        print(t,'Christ.:', res['Ch95'])
        print(t,'Scoring:', )
        print('VaR_99:')
        print(t,' Kupiec:', res['Ku99'])
        print(t,'Christ.:', res['Ch99'])
        print(t,'Scoring:', np.mean(scoring_quantile(r_test, v99, .099)))
        print('EVaR_95:')
        print(t,'Scoring:', np.mean(scoring_expectile(r_test, ev95,.95)))
        print('EVaR_99:')
        print(t,'Scoring:', np.mean(scoring_expectile(r_test, ev99,.99)))
    return(res)


In [10]:
dst = (
"genhyperbolic" ,
'mielke' ,
"johnsonsu" ,
"nct" ,
#'burr' ,
"genlogistic" ,
"norminvgauss" ,
'burr12' ,
#"crystalball" ,
"t" ,
'fisk' ,
"logistic" ,
"tukeylambda" ,
'powernorm' ,
"powerlognorm" ,
"exponweib" ,
"gengamma" ,
"loggamma" ,
"hypsecant" ,
"dweibull" )

results_df=pd.DataFrame()
for dist_type in dst:
    print('')
    print(dist_type)
    filename = './data_distfit/fitted_{}.csv'.format(dist_type)
    df1 = pd.read_csv(filename, index_col=0)
    a = summary_backtesting(df1)
    b =pd.DataFrame.from_dict(a,orient='index',columns=[dist_type])
    results_df = pd.concat([results_df, b.transpose()], ignore_index=False)




genhyperbolic
VaR_95:
         Kupiec: 0.007447710138538399
        Christ.: 0.6492634807020252
        Scoring:
VaR_99:
         Kupiec: 0.18099777733592193
        Christ.: 0.9946240358584997
        Scoring: 0.052556951236776644
EVaR_95:
        Scoring: 6.773175375426059e-05
EVaR_99:
        Scoring: 2.429637994967729e-05

mielke
VaR_95:
         Kupiec: 0.0001276037136812569
        Christ.: 0.839492577936688
        Scoring:
VaR_99:
         Kupiec: 0.18099777733592193
        Christ.: 0.9946240358584997
        Scoring: 0.6739419234923615
EVaR_95:
        Scoring: 0.04115080713496174
EVaR_99:
        Scoring: 0.06137149639673829

johnsonsu
VaR_95:
         Kupiec: 0.007447710138538399
        Christ.: 0.6492634807020252
        Scoring:
VaR_99:
         Kupiec: 0.18099777733592193
        Christ.: 0.9946240358584997
        Scoring: 0.05274791301614226
EVaR_95:
        Scoring: 6.789398912363266e-05
EVaR_99:
        Scoring: 2.443697142778744e-05

nct
VaR_95:
         Kupiec: 0

In [11]:
b = pd.DataFrame.from_dict(a,orient='index',columns=['t'])

In [12]:
results_df

Ku95          Ku99      Ch95      Ch99      Sc95  \
genhyperbolic  7.447710e-03  1.809978e-01  0.649263  0.994624  0.002148   
mielke         1.276037e-04  1.809978e-01  0.839493  0.994624  0.012401   
johnsonsu      7.447710e-03  1.809978e-01  0.649263  0.994624  0.002151   
nct            7.447710e-03  1.809978e-01  0.649263  0.994624  0.002151   
genlogistic    7.447710e-03  1.809978e-01  0.649263  0.994624  0.002162   
norminvgauss   7.447710e-03  1.809978e-01  0.649263  0.994624  0.002154   
burr12         1.276037e-04  1.809978e-01  0.839493  0.994624  0.002199   
t              1.276037e-04  1.438720e-02  0.839493       NaN  0.002266   
fisk           3.219016e-08  1.438720e-02       NaN       NaN  0.047280   
logistic       1.276037e-04  1.438720e-02  0.839493       NaN  0.002284   
tukeylambda    1.276037e-04  1.438720e-02  0.839493       NaN  0.002262   
powernorm      6.270683e-04  5.440159e-01  0.774289  0.906913  0.002183   
powerlognorm   1.563022e-06  1.809978e-01  0.994624  0.994624  0.003890   
exponweib      6.270683e-04  5.440159e-01  0.774289  0.906913  0.002196   
gengamma       3.836292e-04  8.449907e-13  0.027389  0.326252  0.002999   
loggamma       6.270683e-04  5.440159e-01  0.774289  0.906913  0.002192   
hypsecant      1.276037e-04  1.438720e-02  0.839493       NaN  0.002351   
dweibull       6.270683e-04  1.438720e-02  0.774289       NaN  0.002253   

                   Sc99     ESc95     ESc99  
genhyperbolic  0.000594  0.000068  0.000024  
mielke         0.007495  0.041151  0.061371  
johnsonsu      0.000594  0.000068  0.000024  
nct            0.000594  0.000068  0.000024  
genlogistic    0.000609  0.000068  0.000025  
norminvgauss   0.000594  0.000068  0.000024  
burr12         0.000611  0.000069  0.000025  
t              0.000668  0.000074  0.000029  
fisk           0.042945  0.016014  0.003218  
logistic       0.000666  0.000074  0.000029  
tukeylambda    0.000666  0.000073  0.000028  
powernorm      0.000581  0.000068  0.000023  
powerlognorm   0.001195  0.000217  0.000106  
exponweib      0.000581  0.000069  0.000024  
gengamma       0.001387  0.000088  0.000050  
loggamma       0.000581  0.000068  0.000023  
hypsecant      0.000721  0.000079  0.000032  
dweibull       0.000654  0.000072  0.000028

In [15]:
wybrane = np.array(("genhyperbolic", "johnsonsu", 't'))

In [16]:
results_df.transpose()[wybrane]

genhyperbolic  johnsonsu         t
Ku95        0.007448   0.007448  0.000128
Ku99        0.180998   0.180998  0.014387
Ch95        0.649263   0.649263  0.839493
Ch99        0.994624   0.994624       NaN
Sc95        0.002148   0.002151  0.002266
Sc99        0.000594   0.000594  0.000668
ESc95       0.000068   0.000068  0.000074
ESc99       0.000024   0.000024  0.000029

In [17]:
results_df.to_csv('./data/backtesting.csv')

In [18]:
results_df.index

Index(['genhyperbolic', 'mielke', 'johnsonsu', 'nct', 'genlogistic',
       'norminvgauss', 'burr12', 't', 'fisk', 'logistic', 'tukeylambda',
       'powernorm', 'powerlognorm', 'exponweib', 'gengamma', 'loggamma',
       'hypsecant', 'dweibull'],
      dtype='object')